In [1]:
from rdkit.Chem import AllChem as Chem
import molellipsize as mes
import numpy as np
import stk
import glob

In [2]:
from rdkit.Chem.Draw import IPythonConsole

IPythonConsole.ipython_3d = True

import py3Dmol

In [3]:
def show_rdkit_mol(mol):
    data = Chem.MolToMolBlock(mol)
    p = py3Dmol.view(
        data=data,
        style={'stick':{'colorscheme':'cyanCarbon'}}, 
        width=400,
        height=400,
    )
    p.setBackgroundColor('0xeeeeee')
    p.zoomTo()
    p.show()

In [4]:
def show_stk_mol(stk_mol):
    data = Chem.MolToMolBlock(stk_mol.to_rdkit_mol())
    p = py3Dmol.view(
        data=data,
        style={'stick':{'colorscheme':'cyanCarbon'}}, 
        width=400,
        height=400,
    )
    p.setBackgroundColor('0xeeeeee')
    p.zoomTo()
    p.show()

# From RDKit molecules over many conformers

In [5]:
N = 10

In [6]:
smiles = 'c1ccccc1'
rdkitmol = Chem.MolFromSmiles(smiles)
rdkitmol = Chem.AddHs(rdkitmol)
rdkitmol, conformers = mes.ETKDG_UFF_conformers(
    rdkitmol=rdkitmol,
    num_conformers=N,
    randomseed=1000,
)
mes_mol = mes.Molecule(
    rdkitmol=rdkitmol,
    conformers=conformers,
)

In [7]:
show_rdkit_mol(rdkitmol)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [8]:
list(conformers)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [9]:
ellipsoids = mes_mol.get_ellipsoids(
    boxmargin=4.0,
    vdwscale=0.9,
    spacing=0.5,
)

In [10]:
ellipsoids

{0: (array([ 0.02137174, -0.0935038 ,  0.06599361]),
  [3.209338605387556, 6.762540048668652, 7.043816878727093],
  array([[-0.00811844,  0.01287401,  0.99988417],
         [ 0.37074949,  0.92868981, -0.00894709],
         [-0.92869742,  0.37063391, -0.01231254]])),
 1: (array([0.05961066, 0.00659833, 0.02956101]),
  [3.114801938568351, 6.945063214309401, 7.148082700513694],
  array([[-0.0045774 , -0.01639793,  0.99985507],
         [ 0.67882735, -0.73424356, -0.00893411],
         [ 0.73428365,  0.67868807,  0.01449229]])),
 2: (array([ 0.02985845, -0.02407806,  0.0670178 ]),
  [3.2410591722531414, 6.637269982236572, 7.270902279394545],
  array([[-0.01206051,  0.03492713,  0.99931709],
         [ 0.37591487,  0.92623602, -0.02783605],
         [-0.92657571,  0.37532244, -0.02430051]])),
 3: (array([ 0.07806957, -0.03161422,  0.03173174]),
  [3.062449533487999, 7.039221059860702, 7.2835842135653035],
  array([[-0.01173421, -0.02677642,  0.99957257],
         [ 0.94181191,  0.33554215, 

In [11]:
diameters = {i: ellipsoids[i][1] for i in ellipsoids}
mes.plot_diameters(diameters, filename=f'benzene_size.pdf')

In [12]:
ratios = mes_mol.get_inertial_ratios()
mes.plot_shapes(ratios, filename=f'benzene_shape.pdf')

In [13]:
# Pick an ellipsoid to draw.
cid = 0
center, diameter, rotation = ellipsoids[cid]
mes_mol.draw_conformer_hitpoints(
    cid=cid,
    center=center,
    diameter=diameter,
    rotation=rotation,
    boxmargin=4.0,
    vdwscale=0.9,
    spacing=0.5,
    filename=f'benzene_ellhp.pdf',
)
fig, ax = mes.plot_ellipsoid(
    center=center,
    diameter=diameter,
    rotation=rotation,
)
fig.tight_layout()
fig.savefig(f'benzene_ell.pdf', dpi=720, bbox_inches='tight')

## A flexible molecule.

In [27]:
smiles = 'CCCCCC'
rdkitmol = Chem.MolFromSmiles(smiles)
rdkitmol = Chem.AddHs(rdkitmol)
rdkitmol, conformers = mes.ETKDG_UFF_conformers(
    rdkitmol=rdkitmol,
    num_conformers=N,
    randomseed=1000,
)
mes_mol = mes.Molecule(
    rdkitmol=rdkitmol,
    conformers=conformers,
)

In [28]:
show_rdkit_mol(rdkitmol)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [29]:
list(conformers)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [16]:
ellipsoids = mes_mol.get_ellipsoids(
    boxmargin=4.0,
    vdwscale=0.9,
    spacing=0.5,
)
diameters = {i: ellipsoids[i][1] for i in ellipsoids}
ratios = mes_mol.get_inertial_ratios()

In [17]:
mes.plot_shapes(ratios, filename=f'c6_shape.pdf')
mes.plot_diameters(diameters, filename=f'c6_size.pdf')

# From an stk molecule.

In [18]:
# React the amine functional groups during construction.
bb1 = stk.BuildingBlock('NCCN', [stk.PrimaryAminoFactory()])
# React the aldehyde functional groups during construction.
bb2 = stk.BuildingBlock('O=CCCC=O', [stk.AldehydeFactory()])

# Build a polymer.
for i in range(1, 4):
    polymer = stk.ConstructedMolecule(
        topology_graph=stk.polymer.Linear(
            building_blocks=(bb1, bb2),
            repeating_unit='AB',
            num_repeating_units=i,
            optimizer=stk.Collapser(scale_steps=False),
        ),
    )
    mes_mol = mes.Molecule(polymer.to_rdkit_mol(), conformers=[0])
    ellipsoids = mes_mol.get_ellipsoids(
        boxmargin=4.0,
        vdwscale=0.9,
        spacing=0.5,
    )
    diameters = {i: ellipsoids[i][1] for i in ellipsoids}
    print(diameters)

{0: [5.205537759064478, 6.376035040476046, 12.516207344801591]}
{0: [5.451039106837794, 6.734612369757257, 24.607931976435516]}
{0: [5.603325489802962, 6.72926182551461, 38.31910093583731]}


In [19]:
show_stk_mol(polymer)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

# Using arbitrary coordinates to make an rdkit molecule.

In [20]:
# Using a Molecule.
# Fake the positions with H atoms, and some fake bonding.
atom_atomic_nums = [1, 1, 1, 1, 1, 1]
# Atom1 id, atom2 id, bond order.
bonds = [
    (0, 1, 1), 
    (2, 3, 1), 
    (4, 5, 1), 
]
atom_positions = [
    [1, 0, 0], [-1, 0, 0],
    [0, 1, 0], [0, -1, 0],
    [0, 0, 1], [0, 0, -1],
]
# Make rdkit molecule.
mol = Chem.EditableMol(Chem.Mol())
for an in atom_atomic_nums:
    rdkit_atom = Chem.Atom(an)
    mol.AddAtom(rdkit_atom)

for bond in bonds:
    mol.AddBond(
        beginAtomIdx=bond[0],
        endAtomIdx=bond[1],
        order=Chem.BondType(bond[2]),
    )

mol = mol.GetMol()
rdkit_conf = Chem.Conformer(len(atom_atomic_nums))
for atom_id, atom_coord in enumerate(atom_positions):
    rdkit_conf.SetAtomPosition(atom_id, atom_coord)
    mol.GetAtomWithIdx(atom_id).SetNoImplicit(True)
mol.AddConformer(rdkit_conf)

0

In [21]:
show_rdkit_mol(mol)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [22]:
mes_mol = mes.Molecule(mol, [0])
conf_ratios = mes_mol.get_inertial_ratios()
conf_ellipsoids = mes_mol.get_ellipsoids(1.0, 4.0, 0.5)
center = conf_ellipsoids[0][0]
diameters = conf_ellipsoids[0][1]
print(
    f'This fake molecule makes a nice sphere at {center}'
    f' with diameters: {diameters} and inertial ratios '
    f'{conf_ratios[0]}'
)

This fake molecule makes a nice sphere at [ 0.01274316  0.00296594 -0.01053931] with diameters: [4.057776627975807, 4.065560898858799, 4.11301727556738] and inertial ratios (1.0, 1.0)


# Without any molecule!!

In [23]:
# Without a molecule.
points = np.array([
    [1, 0, 0], [-1, 0, 0],
    [0, 1, 0], [0, -1, 0],
    [0, 0, 1], [0, 0, -1],
])
ET = mes.EllipsoidTool()
center, radii, rotation = ET.get_min_vol_ellipse(points)
print(
    f'These points make a nice sphere at {center}'
    f' with radii: {radii}'
)

These points make a nice sphere at [0. 0. 0.] with radii: [1. 1. 1.]


# Load from pore mapper example!

In [24]:
files = glob.glob('../poremapper_video/*_pore.xyz')
print(files)

['../poremapper_video/4D2_C_optc_pore.xyz', '../poremapper_video/5B4_C_optc_pore.xyz', '../poremapper_video/6C1_C_optc_pore.xyz', '../poremapper_video/6A3_C_optc_pore.xyz']


In [25]:
for pore in files:
    pore_name = pore.replace('../poremapper_video/', '')
    # Read XYZ coordinates.
    coordinates = []
    with open(pore, 'r') as f:
        for line in f.readlines()[2:]:
            elem, x, y, z = line.rstrip().split()
            coordinates.append([float(x), float(y), float(z)])
    coordinates = np.array(coordinates)
    
    ET = mes.EllipsoidTool()
    center, radii, rotation = ET.get_min_vol_ellipse(coordinates)
    print(f'{pore_name} with radii: {radii}')
    fig, ax = mes.plot_ellipsoid(
        center=center,
        diameter=radii*2,
        rotation=rotation,
    )
    ax.scatter(
        coordinates[:, 0], coordinates[:, 1], coordinates[:, 2],
        color='r',
        marker='o',
        edgecolor=None,
        s=50,
        alpha=0.2,
    )
    
    fig.tight_layout()
    fig.savefig(f'{pore_name}_ell.pdf', dpi=720, bbox_inches='tight')

4D2_C_optc_pore.xyz with radii: [0.92705598 1.13341291 3.45867085]
5B4_C_optc_pore.xyz with radii: [3.23404401 4.58539335 5.6981495 ]
6C1_C_optc_pore.xyz with radii: [0.09999764 0.10000044 0.1000019 ]
6A3_C_optc_pore.xyz with radii: [0.09999697 0.10000129 0.10000173]
